In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from datetime import datetime
import re

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

In [3]:
data = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')

In [4]:
data['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = data.Rating.mean() # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
sample_submission = pd.read_csv('sample_submission.csv')
df = df_test.append(data, sort=False).reset_index(drop=True)

In [5]:
df.columns = ['Restaurant_id','Name', 'City','Cuisine_Style','Ranking','Price_Range','Num_Rev','Reviews','URL_TA','ID_TA','sample','Rating']

dealing with nans in number of reviews

In [6]:
df['Number_of_Reviews_isNAN'] = pd.isna(df['Num_Rev']).astype('uint8')

dealing with nans in price range

In [7]:
df['no_price_range']=df.Price_Range.apply(lambda x: 0 if x in ['$','$$ - $$$','$$$$'] else 1)

dummy variables for price range

In [8]:
df['cheap']=df.Price_Range.apply(lambda x: 1 if x=='$' else 0)
df['average_price']=df.Price_Range.apply(lambda x: 1 if x=='$$ - $$$' else 0)
df['expencive']=df.Price_Range.apply(lambda x: 1 if x=='$$$$' else 0)

processing price_range

In [9]:
def price_convert1(s):
    if s=='$':
        return 1
    elif s=='$$ - $$$':
        return 2
    elif s=='$$$$':
        return 3
    return 0

In [10]:
df['New_Price_Range']=df.Price_Range.apply(price_convert1)

Filling nans in Number Of Reviews

In [11]:
df.Num_Rev.fillna(0, inplace=True)

since there is different amount of reviews in cities 
//number of reviews - mean for city

In [12]:
city_num_rev = df.groupby(['City'])['Num_Rev'].mean()
df['city_num_rev'] = df.City.apply(lambda x: city_num_rev[x])
df['New_Num_Rev_City']=df.Num_Rev-df['city_num_rev']
df=df.drop(['city_num_rev'], axis=1)

since there is different amount of reviews in price ranges //number of reviews - mean for price range

In [13]:
pr_num_rev = df.groupby(['New_Price_Range'])['Num_Rev'].mean()
df['pr_num_rev'] = df.New_Price_Range.apply(lambda x: pr_num_rev[x])
df['New_Num_Rev_pr']=df.Num_Rev-df.pr_num_rev
df=df.drop(['pr_num_rev'], axis=1)

List of cities, countries and capitals transforming to dummy variables

In [14]:
cities=[]
for s in df.City:
    if s not in cities:
        cities.append(s)

In [15]:
def contain_city(s):
    if city in s:
        return 1
    return 0

In [16]:
for city in cities:
    df[city]=df.City.apply(contain_city)

In [17]:
countries = {'Italy': ['Rome', 'Milan'], 'Germany': ['Berlin', 'Munich', 'Hamburg'], 'Switzerland': ['Geneva', 'Zurich'], 'Portugal': ['Lisbon', 'Oporto'], 'UK': ['London','Edinburgh'], 'Poland': ['Warsaw', 'Krakow'], 'France': ['Paris', 'Lyon'], 'Spain': ['Madrid', 'Barcelona']}

In [18]:
def country(s):
    for i in countries:
        if s in countries[i]:
            return i

In [19]:
df['country']=df.City.apply(country)

In [20]:
df['country'].fillna('one_city', inplace=True)

In [21]:
unique_country=['Italy','Germany','Switzerland','Portugal','UK','Poland','France','Spain']

In [22]:
def dummy_country(s):
    if country in s:
        return 1
    return 0

In [23]:
for country in unique_country:
    df[country]=df.country.apply(dummy_country)

In [24]:
def capitals(s):
    capitals=['Paris', 'Stockholm', 'London', 'Berlin',
       'Bratislava', 'Vienna', 'Rome', 'Madrid',
       'Dublin', 'Brussels', 'Warsaw', 'Budapest', 'Copenhagen',
       'Amsterdam', 'Lisbon', 'Prague', 'Oslo',
       'Helsinki', 'Edinburgh',  'Ljubljana', 'Athens',
       'Luxembourg']
    if s in capitals:
        return 1
    return 0

In [25]:
df['capital']=df.City.apply(capitals)

Eastern or western europe

In [26]:
def easwesteurope(s):
    easteurope = ['Bratislava', 'Warsaw', 'Budapest', 'Prague', 'Ljubljana', 'Krakow']
    if s in easteurope:
        return 1
    return 0  

In [27]:
df['East/West_europe']=df.City.apply(easwesteurope)

Famous tourist cities according to Tripadvisor

In [28]:
def tourist_favorite(s):
    tour_fav=['Paris', 'London', 'Rome', 'Barcelona', 'Prague', 'Lisbon', 'Amsterdam', 'Budapest', 'Berlin', 'Edinburgh', 'Vienna', 'Krakow']
    if s in tour_fav:
        return 1
    return 0 

In [29]:
df['tourist_city']=df.City.apply(tourist_favorite)

Cities with most 5 starred restrants

In [30]:
def most_5(s):
    cities5 = ['Milan', 'Berlin', 'Madrid', 'Paris', 'London', 'Barcelona', 'Rome']
    if s in cities5:
        return 1
    return 0

In [31]:
df['best_rest_cities']=df.City.apply(most_5)

Cities with worst mean star ratings 

In [32]:
def worst4(s):
    worst=['Madrid','Milan','Stockholm','Oslo']
    if s in worst:
        return 1
    return 0

In [33]:
df['worst4']=df.City.apply(worst4)

Adding population from Wiki

In [34]:
populations={'Paris':2140526,
 'Stockholm':972647,
 'London':8908081,
 'Berlin':3748148,
 'Munich':1471508,
 'Oporto':287591,
 'Milan':1395274,
 'Bratislava':424428,
 'Vienna':1888776,
 'Rome':2872800,
 'Barcelona':1620343,
 'Madrid':3223334,
 'Dublin':554554,
 'Brussels':1191604,
 'Zurich':415215,
 'Warsaw':1783321,
 'Budapest':1750268,
 'Copenhagen':1000218,
 'Amsterdam':866737,
 'Lyon':513275,
 'Hamburg':1822445,
 'Lisbon':505526,
 'Prague':1308632,
 'Oslo':690335,
 'Helsinki':650058,
 'Edinburgh':488050,
 'Geneva':201741,
 'Ljubljana':292988,
 'Athens':664046,
 'Luxembourg':613894,
 'Krakow':762508}

In [35]:
def population1(s):
    for i in populations:
        if s == i:
            return populations[i]

In [36]:
df['population']=df.City.apply(population1)

Adding area sizes of cities

In [37]:
area_size= {'London': 1575, 'Paris': 105.4, 'Madrid':  608, 'Barcelona': 100.4, 'Berlin': 3891.9,
           'Milan': 182, 'Rome': 1285.3, 'Prague': 496, 'Lisbon': 83.84, 'Vienna': 414.9, 'Amsterdam':219.3,
                'Munich': 310.4, 'Hamburg': 755.3, 'Brussels': 161.38, 'Stockholm': 414, 'Budapest': 525.1,
                'Warsaw': 517.24, 'Copenhagen':  88.25, 'Dublin': 115, 'Lyon': 47.9, 'Athens': 38,
                'Edinburgh': 259, 'Zurich': 87.9, 'Oporto': 41.3, 'Geneva': 15.9, 'Krakow': 326.8, 
                'Helsinki': 213.75, 'Oslo': 454, 'Bratislava': 367.6, 'Luxembourg': 51.7, 'Ljubljana': 275}

In [38]:
def area(s):
    for i in area_size:
        if s == i:
            return area_size[i]

In [39]:
df['area_size']=df.City.apply(area)

Preparation function for cusines and fiiling no cusine 

In [40]:
def list_prep(s):
    s=s[2:-2]
    return s

In [41]:
df.Cuisine_Style=df.Cuisine_Style.fillna("['no_cusine']")

getting set of unique cusines and processing them to dummy variables

In [42]:
un_cus=[]
def unique_cusine(s):
    s=s.split(sep='\', \'')
    for x in range(len(s)):
        if s[x] not in un_cus:
            un_cus.append(s[x])

In [43]:
df.Cuisine_Style=df.Cuisine_Style.apply(list_prep)

In [44]:
df.Cuisine_Style.apply(unique_cusine)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
49970    None
49971    None
49972    None
49973    None
49974    None
49975    None
49976    None
49977    None
49978    None
49979    None
49980    None
49981    None
49982    None
49983    None
49984    None
49985    None
49986    None
49987    None
49988    None
49989    None
49990    None
49991    None
49992    None
49993    None
49994    None
49995    None
49996    None
49997    None
49998    None
49999    None
Name: Cuisine_Style, Length: 50000, dtype: object

In [45]:
def contain_cusine(s):
    if cusine in s:
        return 1
    return 0

In [46]:
for cusine in un_cus:
    df[cusine]=df.Cuisine_Style.apply(contain_cusine)

Number of cusines in resturant and dummying it a little bit

In [47]:
def cusines_in_rest(s):

    if 'no_cusine' in s:
        return 0
    return len(s)
df['cusine_len']=df.Cuisine_Style.apply(cusines_in_rest)

In [48]:
def one_cusine(s):
    if s==1:
        return 1
    return 0

In [49]:
def two_cusines(s):
    if s==2:
        return 1
    return 0

In [50]:
def threeormore_cusines(s):
    if s>=3:
        return 1
    return 0 

In [51]:
df['One_cusine']=df.cusine_len.apply(one_cusine)
df['two_cusines']=df.cusine_len.apply(two_cusines)
df['threeormore_cusines']=df.cusine_len.apply(threeormore_cusines)

Local food dummies

In [52]:
def localfood(s):
    if s>=2:
        return 1
    return 0

In [53]:
df['Am_Local']=df['Amsterdam']+df['Dutch']
df['Athens_local']=df.Athens+df.Greek
df['Brussels_local']=df.Brussels+df.Belgian
df['Budapest_local']=df.Budapest+df.Hungarian
df['Copenhagen_local']=df.Copenhagen+df.Danish
df['Dublin_local']=df.Dublin+df.Irish
df['Edinburgh_local']=df.Edinburgh+df.Scottish
df['London_local']=df.London+df.British
df['Italy_local']=df.Italy+df.Italian
df['Germany_local']=df.Germany+df.German
df['Portugal_local']=df.Portugal+df.Portuguese
df['Poland_local']=df.Poland+df.Polish
df['France_local']=df.France+df.French
df['Ljubljana_local']=df.Ljubljana+df.Slovenian
df['Oslo_local']=df.Oslo+df.Norwegian
df['Prague_local']=df.Prague+df.Czech
df['Stockholm_local']=df.Stockholm+df.Swedish
df['Vienna_local']=df.Vienna+df.Austrian
df['Switzerland_local']=df.Switzerland+df.Swiss


df.Am_Local=df.Am_Local.apply(localfood)
df['Athens_local']=df['Athens_local'].apply(localfood)
df['Brussels_local']=df['Brussels_local'].apply(localfood)
df['Budapest_local']=df['Budapest_local'].apply(localfood)
df['Copenhagen_local']=df['Copenhagen_local'].apply(localfood)
df['Dublin_local']=df['Dublin_local'].apply(localfood)
df['Edinburgh_local']=df['Edinburgh_local'].apply(localfood)
df['London_local']=df['London_local'].apply(localfood)
df['Italy_local']=df['Italy_local'].apply(localfood)
df['Germany_local']=df['Germany_local'].apply(localfood)
df['Portugal_local']=df['Portugal_local'].apply(localfood)
df['Poland_local']=df['Poland_local'].apply(localfood)
df['France_local']=df['France_local'].apply(localfood)
df['Ljubljana_local']=df['Ljubljana_local'].apply(localfood)
df['Oslo_local']=df['Oslo_local'].apply(localfood)
df['Prague_local']=df['Prague_local'].apply(localfood)
df['Stockholm_local']=df['Stockholm_local'].apply(localfood)
df['Vienna_local']=df['Vienna_local'].apply(localfood)
df['Switzerland_local']=df['Switzerland_local'].apply(localfood)

starting with reviews / nans and dummying nans

In [54]:
df['Reviews'].fillna('[[], []]', inplace=True)

In [55]:
df['No_Reviews']=df['Reviews'].apply(lambda x: 1 if x=='[[], []]' else 0)

In [56]:
def list_prep1(s):
    s=s[3:-3]
    return s

In [57]:
df['Reviews']=df['Reviews'].apply(list_prep1)

getting dates data 

In [58]:
def get_last_unix_date(s):
    if s=='], [':
        return 0
    dates = s.split('], [')
    if len(dates)>1:
        mask = '%m/%d/%Y'
        result = re.findall('\d+/\d+/\d+', dates[1])
        date1=datetime.strptime(result[0], mask )
        return datetime.timestamp(date1)
    return 0

In [59]:
df['last_comment_date']=df['Reviews'].apply(get_last_unix_date)

In [60]:
def between_dates(s):
    if s=='], [':
        return 0
    dates = s.split('], [')
    if len(dates)>1:
        mask = '%m/%d/%Y'
        result = re.findall('\d+/\d+/\d+', dates[1])
        date1=datetime.strptime(result[0], mask )
        if len(result)>1:
            date2=datetime.strptime(result[1], mask )
            between_dates=date1-date2
            return float(abs(between_dates.days))
        return 0
    return 0

In [61]:
df['between_dates']=df['Reviews'].apply(between_dates)

preparing reviews for simple word analisys

In [62]:
def rew_prep(s):
    if s=='], [':
        return nan
    s=s.lower()
    s= re.sub(r'[^a-zA-Z\s]', '', s)
    return s

In [63]:
df['Reviews']=df['Reviews'].apply(rew_prep)

I've made the vocabularies of words for every rating and separeted related words. 

In [64]:
voc1=['worst', 'terrible', 'poor','bad', 'horrible', 'awful', 'avoid', 'rude', 'disappointing','worse'
      ,'disaster','nasty','disappointed','dirty','horrific' ,
       'unhygienic', 'hostile', 'cheated', 'dreadful',  'rudely', 'waste', 'unpleasant', 'garbage', 'fake']

In [65]:
def vocab1(s):
    count=0
    for x in voc1:
        if x in s:
            count+=1
    return count   

In [66]:
df['Voc1']=df['Reviews'].apply(vocab1)

In [67]:
voc15=['avoid', 'worst', 'expensive', 'rip', 'bad', 'terrible',  'awful', 'disgusting', 'trap', 'horrible', 'overpriced'
       , 'poor','ripped', 'rude', 'disappointing','scam','mediocre','shame', 'dreadful','disappointed', 'dishonest', 
       'nightmare', 'poison','undercooked','inefficient', 'disgraceful' , 'racist', 'thieves','awfull'
     , 'cremated', 'horrific', 'poisoned' ]

In [68]:
def vocab15(s):
    count=0
    for x in voc15:
        if x in s:
            count+=1
    return count   

In [69]:
df['Voc15']=df['Reviews'].apply(vocab15)

In [70]:
voc2=[ 'bad', 'poor', 'worst', 'avoid', 'terrible','horrible', 'average','rude', 'expensive', 'disappointing', 
       'trap','awful', 'overpriced','slow','disappointed','excellent', 'rip','disappointment', 
       'scam','dreadful', 'disgusting' , 'passable', 'unhelpful', 'rubish' , 'poorly', 'illegal', 'medicore', 'cheater' ]

In [71]:
def vocab2(s):
    count=0
    for x in voc2:
        if x in s:
            count+=1
    return count 

In [72]:
df['Voc2']=df['Reviews'].apply(vocab2)

In [73]:
voc25=[ 'good', 'not', 'great', 'nice', 'very', 'average', 'poor', 'worst', 'avoid', 'disappointing', 'dont', 'terrible',
       'better', 'awful', 'rude', 'expensive', 'friendly','cheap', 'quality', 'excellent', 'decent' ,'overpriced', 'horrible',
       'little', 'tasty', 'delicious', 'slow', 'busy', 'small', 'enough', 'disappointed' ]

In [74]:
def vocab25(s):
    count=0
    for x in voc25:
        if x in s:
            count+=1
    return count 

In [75]:
df['Voc25']=df['Reviews'].apply(vocab25)

In [76]:
voc3=[ 'good', 'great', 'nice', 'very', 'average', 'bad', 'best', 'quick', 'poor', 'worst', 'excellent', 'friendly', 'terrible'
       , 'cheap', 'expensive', 'lovely', 'bar', 'decent', 'better', 'tasty', 'slow', 'disappointing', 'fast', 'avoid', 'rude'
       'really', 'little', 'quality', 'awful', 'pub', 'overpriced', 'perfect', 'horrible', 'away', 'delicious', 'worth', 'typical',
       'reasonable', 'special', 'mediocre', 'small', 'amazing', 'family', 'pricey', 'big', 'well', 'pleasant', 'never', 'busy'
       , 'authentic', 'friends', 'disappointed', 'fantastic', 'traditional', 'beautiful', 'fresh', 'hot', 'cosy', 'love'
       , 'trap', 'low', 'worse', 'dirty', 'unfriendly', 'cheerful', 'disappointment', 'fine']

In [77]:
def vocab3(s):
    count=0
    for x in voc3:
        if x in s:
            count+=1
    return count 

In [78]:
df['Voc3']=df['Reviews'].apply(vocab3)

In [79]:
voc35=['good',  'great','nice', 'very', 'average', 'excellent', 'best', 'tasty', 'friendly', 'delicious'
       ,'lovely','quick','bad', 'atmosphere','cheap', 'bar', 'decent', 'little', 'local', 'pub', 'quality', 'better', 
       'poor', 'expensive', 'beer', 'authentic','amazing', 'fast', 'disappointing', 'perfect', 'reasonable', 'pleasant'
       'terrible', 'fantastic', 'slow', 'worst', 'wonderful', 'fun', 'cosy', 'late', 'simple', 'traditional',
       'overpriced', 'rude', 'okay', 'hidden', 'gem','cozy' , 'typical', 'super'
       , 'beautiful', 'quiet', 'awful', 'avoid', 'easy', 'pretty', 'awesome', 'love', 'pricey', 'real' , 'horrible',
        'cool', 'enjoyable', 'fair', 'large', 'high', 'happy', 'clean', 'affordable', 'disappointed', 'mediocre',
        'recommended', 'yummy', 'expected', 'convenient', 'surprisingly' , 'relaxed', 'fabulous', 'cute', 
        'reasonably', 'unfriendly', 'welcoming', 'helpful'
       , 'usual', 'fare', 'low', 'brilliant', 'outstanding', 'honest', 'favourite', 'charming', 'relaxing', 'variety', 'wow'
       , 'absolutely', 'popular', 'comfortable', 'vegan', 'light', 'inexpensive', 'wrong', 'cheerful', 'quaint', 'surprised', 'ideal', 
       'free', 'meals', 'shame', 'yum', 'incredible', 'vibe', 'rather', 'slightly', 'middle', 'heaven', 'huge', 'fairly', 'recommend'
       , 'unique', 'gorgeous', 'boring', 'crowded', 'solid', 'perfectly', 'impressed', 'acceptable', 'overall', 'noisy', 
       'stylish', 'extremely', 'needed', 'almost', 'waste', 'quirky', 'reliable', 'efficient', 'generous', 'disappointment'
       , 'dirty', 'expectations', 'improvement', 'highly', 'unpretentious', 'underwhelming', 'less', 'delightful']

In [80]:
def vocab35(s):
    count=0
    for x in voc35:
        if x in s:
            count+=1
    if count>=1:
        return 1
    return count

In [81]:
df['Voc35']=df['Reviews'].apply(vocab35)

In [82]:
voc4=['good', 'great', 'nice', 'very', 'excellent', 'best', 'delicious', 'tasty', 'lovely', 'pizza', 'friendly', 'italian', 'local', 'little'
       ,  'bar', 'amazing', 'quick', 'authentic', 'pub', 'sushi', 'coffee', 'really', 'cheap', 'perfect', 'fantastic',
       'fresh', 'average', 'decent', 'beer', 'tapas', 'wonderful', 'pleasant', 'small', 'worth', 'bad', 'eat', 'traditional', 'gem', 'fast', 'expensive', 
       'reasonable' 'burger', 'ok', 'wine', 'fish', 'so', 'cozy', 'indian', 'simple', 'family', 'view', 'cosy', 
         'chinese', 'better' 'drink', 'surprise', 'fun', 'hidden', 'beautiful', 'like', 'ever', 'japanese'
       , 'love', 'old', 'well', 'awesome', 'poor', 'new', 'pasta', 'seafood', 'interesting', 'cool', 'typical',
        'slow', 'quiet', 'yummy', 'healthy', 'relaxed', 'friends', 'disappointing', 'fine', 
        'affordable', 'classic', 'fabulous', 'different', 'overpriced',  'enjoyable', 
        'charming', 'pretty', 'modern', 'happy', 'welcoming'] 

In [83]:
def vocab4(s):
    count=0
    for x in voc4:
        if x in s:
            count+=1
    return count

In [84]:
df['Voc4']=df['Reviews'].apply(vocab4)

In [85]:
voc45=[ 'great', 'good', 'best', 'excellent', 'nice', 'very', 'delicious', 'amazing', 'friendly', 'lovely',
      'tasty', 'fantastic', 'little', 'bar', 'wonderful', 'atmosphere' , 'perfect', 'authentic', 'gem', 'local', 
        'fresh', 'hidden', 'wine',  'awesome', 'cozy', 'small', 'cheap', 'cosy',  'superb', 'quick', 'super'
       , 'family', 'love', 'traditional', 'worth', 'surprise', 'fabulous' , 'yummy', 'brilliant', 'beautiful', 'top', 'favourite',  'cool', 'outstanding'
       , 'pleasant', 'simple', 'well', 'loved', 'near', 'favorite', 'wow', 'better' , 'decent', 'healthy', 'absolutely', 'fast', 'recommended', 'highly' , 'cute', 'quiet', 'typical', 'reasonable', 'warm'
       , 'interesting'
       , 'relaxed', 'delightful', 'incredible', 'expensive', 'welcoming', 'relaxing', 'homemade', 'affordable']

In [86]:
def vocab45(s):
    count=0
    for x in voc45:
        if x in s:
            count+=1
    return count

In [87]:
df['Voc45']=df['Reviews'].apply(vocab45)

In [88]:
voc5=['great', 'best', 'good','excellent', 'amazing', 'nice',  'delicious', 'with','very', 'little', 'friendly', 'lovely', 'coffee',
      'gem',  'bar', 'fantastic', 'tasty', 'authentic', 'fresh', 'perfect', 'wonderful', 'hidden', 'local', 'wine', 'new','love',
      'super', 'awesome',  'fabulous', 'cosy', 'beer', 'cozy',
      'small','yummy', 'beautiful', 'highly','surprise','absolutely', 'favourite', 'top','brilliant','recommended','pleasant']

In [89]:
def vocab5(s):
    count=0
    for x in voc5:
        if x in s:
            count+=1
    return count

In [90]:
df['Voc5']=df['Reviews'].apply(vocab5)

Since Ranking in dataset is ranking of resturant in particular city,  i decided to make a more realistic ranking for model

In [91]:
city_mean = df.groupby(['City'])['Ranking'].mean()
city_max = df.groupby(['City'])['Ranking'].max()
df['rankingmaxcity'] = df['City'].apply(lambda x: city_max[x])
city_count= df['City'].value_counts(ascending=False)
df['rankingmeancity'] = df['City'].apply(lambda x: city_mean[x])
df['rankingincity'] = df['City'].apply(lambda x: city_count[x])
df['newranking'] = (df['Ranking'] - df['rankingmeancity']) / df['rankingincity']
df['NewRank1']=df['rankingmaxcity']-df['Ranking']

From Tripadvisor cite , we know, that ranking strongly related to number of reviews and relevancy of dates for reviews. https://www.tripadvisor.com/TripAdvisorInsights/w765

In [92]:
def formula_trip(row):
    if int(row['Num_Rev'])>50 and row['last_comment_date']> 1503705600:
        return 1
    return 0

In [93]:
df['formula_trip']=df.apply(formula_trip, axis=1)

In [94]:
df['Ranking_Nr_Const']=df.Num_Rev*df['Ranking']
df['Ranking_Last_Comment_Const']=df['Ranking']*df['last_comment_date']

Amount of resturants for population may contribute to model a bit 

In [95]:
df['rest_per_population']=df.population/df.rankingincity

In [96]:
df=df.drop(['rankingmeancity','rankingincity'], axis=1)

Now we need to scale the data to avoid overrepresenting

In [97]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

norm=df[['Ranking','Num_Rev','New_Price_Range','cusine_len','between_dates','newranking','population','area_size','rest_per_population','New_Num_Rev_City','New_Num_Rev_pr','Voc1','Voc15','Voc2','Voc25','Voc3','Voc35','Voc45','Voc5','last_comment_date','Voc4','Ranking_Nr_Const','Ranking_Last_Comment_Const','NewRank1','rankingmaxcity']]
scaler = StandardScaler() 
df_scaled = scaler.fit_transform(norm)
df[['Ranking','Num_Rev','New_Price_Range','cusine_len','between_dates','newranking','population','area_size','rest_per_population','New_Num_Rev_City','New_Num_Rev_pr','Voc1','Voc15','Voc2','Voc25','Voc3','Voc35','Voc45','Voc5','last_comment_date','Voc4','Ranking_Nr_Const','Ranking_Last_Comment_Const','NewRank1','rankingmaxcity']]=df_scaled

In [98]:
df=df.drop(['Restaurant_id','City','Name','Cuisine_Style','Price_Range','Reviews','URL_TA','ID_TA','country','New_Price_Range'], axis=1)

df=df.fillna(0)

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)


y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [101]:
train_data.shape, X.shape, X_train.shape, X_test.shape

((40000, 224), (40000, 223), (32000, 223), (8000, 223))

In [102]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [103]:
# Создаём модель

model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных

model.fit(X_train, y_train)
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred

y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   23.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [104]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.19654749999999999


In [105]:
test_data = df.query('sample == 0').drop(['sample'], axis=1)
test_data = test_data.drop(['Rating'], axis=1)

In [106]:
predict_submission = model.predict(test_data)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


We need to round our results for better predicting

In [107]:
def round_half(num):
    if num < 1.25:
        num = 1.0
    elif num < 1.75:
        num = 1.5
    elif num < 2.25:
        num = 2
    elif num < 2.75:
        num = 2.5
    elif num < 3.25:
        num = 3.0
    elif num < 3.75:
        num = 3.5
    elif num < 4.25:
        num = 4.0
    elif num < 4.75:
        num = 4.5
    else:
        num = 5.0
    return num

In [108]:
sample_submission['Rating'] = predict_submission
sample_submission['Rating']=sample_submission['Rating'].apply(round_half)
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

,Restaurant_id,Rating
0,id_0,3.0
1,id_1,4.0
2,id_2,4.5
3,id_3,4.5
4,id_4,4.5
